# Midterm
Cesar Godoy

In [1]:
# Midterm
#Cesar Godoy
print("Midterm \nCesar Godoy")

Midterm 
Cesar Godoy


## Introduction

The Brazilian government provides plenty of data about the activities of congress - especially the house of representatives. I was interested on looking into the information they provide and thinking about developing systems or data analytics tools that would give normal citizens accurate information on their representatives and allow democracy to work with less assymetry of information.

In this project I will use a mix of government APIs and csv files to create a facts table about voting in the House of Representatives in Brazil.

In [2]:
# import packages
import requests
import pandas as pd
import pymongo
import os
from sqlalchemy import create_engine, text
import re

## Data Origins and Extraction
In this section, I will:
(1) Insert data in MongoDB and MySQL, to mimic that being their origin
(2) Extract data from (2.1) an API (2.2) my local MongoDB (2.3) my local file system (2.4) my local MySQL server

### API
Swagger Documentation: https://dadosabertos.camara.leg.br/swagger/api.htmls
In this subsection, I will:
(1) retrieve data from an API endpoint
(2) save it as a DataFrame

In [3]:
# additional functions for this section
# make requests
def requireAPIs(url):
    try:
        req = requests.get(url)
        return req.json()
    except: 
        print(f"couldn't read {url}")
        return False
#create urls
base = "https://dadosabertos.camara.leg.br/api/v2"
partidos_url = base+"//partidos?itens=100"


In [4]:
# dataframe about the votes taken
# they probably limit access to the API - this section takes a few minutes to run!
condition = True
votacoes_df = pd.DataFrame()
votacoes_url = base+"/votacoes?dataInicio=2022-01-01&dataFim=2022-12-31&pagina=1"
while(condition):
    print(votacoes_url)
    votacoes_json = requireAPIs(votacoes_url)
    votacoes_url = votacoes_json["links"][1]['href']
    condition = votacoes_json["links"][1]['rel']=="next"
    votacoes_intermediary = pd.DataFrame(data=votacoes_json['dados'])
    votacoes_df = pd.concat([votacoes_df,votacoes_intermediary],ignore_index=True)
    if(not condition): print("done!")
print(len(votacoes_df.index))

https://dadosabertos.camara.leg.br/api/v2/votacoes?dataInicio=2022-01-01&dataFim=2022-12-31&pagina=1
https://dadosabertos.camara.leg.br/api/v2/votacoes?dataInicio=2022-01-01&dataFim=2022-12-31&pagina=2&itens=200
https://dadosabertos.camara.leg.br/api/v2/votacoes?dataInicio=2022-01-01&dataFim=2022-12-31&pagina=3&itens=200
https://dadosabertos.camara.leg.br/api/v2/votacoes?dataInicio=2022-01-01&dataFim=2022-12-31&pagina=4&itens=200
https://dadosabertos.camara.leg.br/api/v2/votacoes?dataInicio=2022-01-01&dataFim=2022-12-31&pagina=5&itens=200
https://dadosabertos.camara.leg.br/api/v2/votacoes?dataInicio=2022-01-01&dataFim=2022-12-31&pagina=6&itens=200
https://dadosabertos.camara.leg.br/api/v2/votacoes?dataInicio=2022-01-01&dataFim=2022-12-31&pagina=7&itens=200
https://dadosabertos.camara.leg.br/api/v2/votacoes?dataInicio=2022-01-01&dataFim=2022-12-31&pagina=8&itens=200
https://dadosabertos.camara.leg.br/api/v2/votacoes?dataInicio=2022-01-01&dataFim=2022-12-31&pagina=9&itens=200
https://dad

In [13]:
votacoes_df.head(1)

,id,uri,data,dataHoraRegistro,siglaOrgao,uriOrgao,uriEvento,proposicaoObjeto,uriProposicaoObjeto,descricao,aprovacao
0,2307143-25,https://dadosabertos.camara.leg.br/api/v2/vota...,2022-12-14,2022-12-23T17:52:27,CE,https://dadosabertos.camara.leg.br/api/v2/orga...,https://dadosabertos.camara.leg.br/api/v2/even...,None,None,Aprovado o Relatório Prévio.,1.0


### MongoDB
In this section:
(1) Retrieve data from an API endpoit
(2) Save data in MongoDB (to mimic MongoDB being the original source of data)
(3) Extract data from MongoDB
(4) Convert it into a Pandas DataFrame

In [14]:
# additional functions for this section
def get_mongo_dataframe(connect_str, db_name, collection, query):
    '''Create a connection to MongoDB'''
    client = pymongo.MongoClient(connect_str)
    
    '''Query MongoDB, and fill a python list with documents to create a DataFrame'''
    db = client[db_name]
    dframe = pd.DataFrame(list(db[collection].find(query)))
    dframe.drop(['_id'], axis=1, inplace=True)
    client.close()
    return dframe

#### Steps 1 and 2: Fetch data from api and save it in MongoDB

In [15]:
#connect to mongoDB
atlas_cluster_name = "sandbox.zibbf"
atlas_user_name = "root_user"
atlas_password = "Eutocomfome,meuamigo!"

conn_str = {"local" : f"mongodb://localhost:27017/",
    "atlas" : f"mongodb+srv://{atlas_user_name}:{atlas_password}@{atlas_cluster_name}.mongodb.net"
}

print(f"Local Connection String: {conn_str['local']}")

Local Connection String: mongodb://localhost:27017/


In [16]:
#push data to mongoDB
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
db = myclient["local"]
db.drop_collection("partidos")
partidos_json = requireAPIs(partidos_url)['dados']
if(partidos_json): db["partidos"].insert_many(partidos_json)
else: print("failed to save partidos to MongoDB")
myclient.close()        

#### Steps 3 and 4: Extract data from MongoDB and convert it into a Pandas DataFrame

In [17]:
# dataframe about party information
partidos_df = get_mongo_dataframe(conn_str["local"],"local","partidos",{})
partidos_df.head(1)

,id,sigla,nome,uri
0,36898,AVANTE,Avante,https://dadosabertos.camara.leg.br/api/v2/part...


### File System
In this section, I will:
(1) Retrieve data from my local file system
(2) Save it as a DataFrame

In [18]:
# dataframe about departments
orgaos_df = pd.read_csv("data-used/orgaos.csv", delimiter=";")
orgaos_df.head(1)

,uri,sigla,apelido,nome,nomePublicacao,codTipoOrgao,tipoOrgao,dataInicio,dataInstalacao,dataFim,dataFimOriginal,codSituacao,descricaoSituacao,casa,sala,urlWebsite
0,https://dadosabertos.camara.leg.br/api/v2/orga...,MESA,Mesa Diretora,Mesa Diretora da Câmara dos Deputados,Mesa Diretora,1,Comissão Diretora,NaN,NaN,NaN,NaN,NaN,NaN,Câmara dos Deputados,NaN,NaN


### MySQL
In this section I will:
(1) Retrieve data from my local file system
(2) Save data in MySQL (to mimic MySQL being the original source of data)
(3) Extract data from MySQL
(4) Convert it into a Pandas DataFrame

In [19]:
#additional functions for this section
#connect to mysql
host_name = "localhost"
host_ip = "127.0.0.1"
port = "3306"
user_id = "root_user"
pwd = "Eutocomfome,meuamigo!"
mysql_db_src = "DadosAbertos_sql_src"
mysql_db_dst = "DadosAbertos_sql_dst"
def createDatabase(database):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/sys" #use default sys db to create db I'll use
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    with sqlEngine.connect() as conn:
        conn.execute(text(f"DROP DATABASE IF EXISTS `{database}`;"))
        conn.execute(text(f"CREATE DATABASE `{database}`;"))
        conn.execute(text(f"USE {database};"))
    
def set_dataframe2(df,database,table_name, db_operation,pk_column):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{database}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    with sqlEngine.connect() as conn:
        if db_operation == "insert": 
            df.to_sql(table_name, con=conn, index=False, if_exists='replace')
            conn.execute(text(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column})"))
        elif db_operation == "update": 
            df.to_sql(table_name, con=conn, index=False, if_exists='append')

def get_dataframe(user_id, pwd, host_name, db_name, sql_query):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    dframe = pd.read_sql(text(sql_query), connection);
    connection.close()
    return dframe

Run SQL command that creates dim_date at this point.

In [21]:
dim_date = get_dataframe(user_id,pwd,host_name,mysql_db_dst, "SELECT * FROM dim_date")
dim_date = pd.DataFrame(dim_date, columns=["date_key","full_date"])
dim_date["full_date"] = dim_date["full_date"].astype('datetime64[ns]')
dim_date.dtypes

date_key              int64
full_date    datetime64[ns]
dtype: object

#### Steps 1 and 2: Retrieve data from my local file system and Save data in MySQL

In [22]:
createDatabase(mysql_db_src)

In [23]:
votacoesVotos_df_in = pd.read_csv("data-used/votacoesVotos-2022.csv", delimiter=";")
votacoesVotos_df_in['votos_key'] = votacoesVotos_df_in.index
votacoesVotos_df_in.head(1)

,idVotacao,uriVotacao,dataHoraVoto,voto,deputado_id,deputado_uri,deputado_nome,deputado_siglaPartido,deputado_uriPartido,deputado_siglaUf,deputado_idLegislatura,deputado_urlFoto,votos_key
0,105464-319,https://dadosabertos.camara.leg.br/api/v2/vota...,2022-08-04T13:19:31,Não,204521,https://dadosabertos.camara.leg.br/api/v2/depu...,Abou Anni,UNIÃO,https://dadosabertos.camara.leg.br/api/v2/part...,SP,56,https://www.camara.leg.br/internet/deputado/ba...,0


In [24]:
set_dataframe2(votacoesVotos_df_in,mysql_db_src,"votos","insert","votos_key")

#### Steps 3 and 4:Extract data from MySQL and Convert it into a Pandas DataFrame

In [25]:
# dataframe about how each person voted
votacoesVotos_df = get_dataframe(user_id,pwd,host_name,mysql_db_src, "SELECT * FROM votos")
votacoesVotos_df.head(1)

,idVotacao,uriVotacao,dataHoraVoto,voto,deputado_id,deputado_uri,deputado_nome,deputado_siglaPartido,deputado_uriPartido,deputado_siglaUf,deputado_idLegislatura,deputado_urlFoto,votos_key
0,105464-319,https://dadosabertos.camara.leg.br/api/v2/vota...,2022-08-04T13:19:31,Não,204521,https://dadosabertos.camara.leg.br/api/v2/depu...,Abou Anni,UNIÃO,https://dadosabertos.camara.leg.br/api/v2/part...,SP,56,https://www.camara.leg.br/internet/deputado/ba...,0


## Transform and Load
In this section I will:
(1) Transform the data by removing columns and changing rows
(2) Create dimension tables
(3) Load data into (a different) MySQL (database)

In [26]:
#additional functions for this section
def replaceURIwithID(df, field):
    return df[field].apply(lambda x: None if x is None else re.findall(r'[^/]+$', str(x))[0])
def getDate_from_DataHoraVoto(df, field):
    return df[field].apply(lambda x: None if x is None else re.findall(r'\d{4}-\d{2}-\d{2}', str(x))[0])
def getTime_from_DataHoraVoto(df, field):
    return df[field].apply(lambda x: None if x is None else re.findall(r'\d{2}:\d{2}:\d{2}', str(x))[0])


#### Step 1 and 2: Transform the data by removing columns and changing rows and Create dimension tables

In [27]:
votacoes_df["idOrgao"] = replaceURIwithID(votacoes_df,"uriOrgao")
votacoes_df["descricaoVotacao"] = votacoes_df["descricao"]
votacoes_df["idVotacao"] = replaceURIwithID(votacoes_df,"uri")
votacoes_dim = pd.DataFrame(votacoes_df, 
                            columns=["idVotacao", "data", "idOrgao", 
                                     "descricaoVotacao", "aprovacao"])
votacoes_dim["data"] = votacoes_dim["data"].astype('datetime64[ns]')
votacoes_dim["votacoes_key"] = votacoes_dim.index
votacoes_dim = votacoes_dim.merge(dim_date, how="left", left_on="data", right_on="full_date")
votacoes_dim = votacoes_dim.drop(["data","full_date"], axis=1)
votacoes_dim["data"] = votacoes_dim["date_key"]
votacoes_dim = votacoes_dim.drop(["date_key"], axis=1)
votacoes_dim.head(1)

,idVotacao,idOrgao,descricaoVotacao,aprovacao,votacoes_key,data
0,2307143-25,2009,Aprovado o Relatório Prévio.,1.0,0,20221214


In [28]:
partidos_df['deputado_nomePartido'] = partidos_df["nome"]
partidos_dim = pd.DataFrame(partidos_df,columns=["deputado_nomePartido","sigla"])
partidos_dim["partidos_key"] = partidos_dim.index
partidos_dim.head(1)

,deputado_nomePartido,sigla,partidos_key
0,Avante,AVANTE,0


In [29]:
orgaos_df["idOrgao"] = replaceURIwithID(orgaos_df,"uri")
orgaos_dim = pd.DataFrame(orgaos_df,columns=["idOrgao","sigla", "apelido","tipoOrgao"])
orgaos_dim["orgao_key"] = orgaos_dim.index
orgaos_dim.head(1)

,idOrgao,sigla,apelido,tipoOrgao,orgao_key
0,4,MESA,Mesa Diretora,Comissão Diretora,0


In [30]:
votacoesVotos_df["dataVoto"] = getDate_from_DataHoraVoto(votacoesVotos_df,"dataHoraVoto").astype('datetime64[ns]')
votacoesVotos_df["horaVoto"] = getTime_from_DataHoraVoto(votacoesVotos_df,"dataHoraVoto")
votos_dim = pd.DataFrame(votacoesVotos_df,
                                 columns=["idVotacao","dataVoto","horaVoto", "voto","deputado_id",
                                          "deputado_nome","deputado_siglaPartido","deputado_siglaUf"])
votos_dim["votos_key"] = votos_dim.index
votos_dim = votacoesVotos_df.merge(dim_date, how="left", left_on="dataVoto", right_on="full_date")
votos_dim = votos_dim.drop(["dataVoto","full_date"], axis=1)
votos_dim["dataVoto"] = votos_dim["date_key"]
votos_dim = votos_dim.drop(["date_key"], axis=1)
votos_dim.head(1)

,idVotacao,uriVotacao,dataHoraVoto,voto,deputado_id,deputado_uri,deputado_nome,deputado_siglaPartido,deputado_uriPartido,deputado_siglaUf,deputado_idLegislatura,deputado_urlFoto,votos_key,horaVoto,dataVoto
0,105464-319,https://dadosabertos.camara.leg.br/api/v2/vota...,2022-08-04T13:19:31,Não,204521,https://dadosabertos.camara.leg.br/api/v2/depu...,Abou Anni,UNIÃO,https://dadosabertos.camara.leg.br/api/v2/part...,SP,56,https://www.camara.leg.br/internet/deputado/ba...,0,13:19:31,20220804


In [31]:
print(len(votos_dim.index))

197189


In [32]:
votacoes_orgaos = votacoes_dim.merge(orgaos_dim, how="left", on="idOrgao")
votacoes_orgaos.head(1)

,idVotacao,idOrgao,descricaoVotacao,aprovacao,votacoes_key,data,sigla,apelido,tipoOrgao,orgao_key
0,2307143-25,2009,Aprovado o Relatório Prévio.,1.0,0,20221214,CE,Educação,Comissão Permanente,19


In [33]:
print(len(votacoes_orgaos.index))

5578


In [34]:
votos_partidos = votos_dim.merge(partidos_dim, how="left", left_on="deputado_siglaPartido",right_on="sigla")
votos_partidos=votos_partidos.drop(columns=["sigla"])
votos_partidos.head(1)

,idVotacao,uriVotacao,dataHoraVoto,voto,deputado_id,deputado_uri,deputado_nome,deputado_siglaPartido,deputado_uriPartido,deputado_siglaUf,deputado_idLegislatura,deputado_urlFoto,votos_key,horaVoto,dataVoto,deputado_nomePartido,partidos_key
0,105464-319,https://dadosabertos.camara.leg.br/api/v2/vota...,2022-08-04T13:19:31,Não,204521,https://dadosabertos.camara.leg.br/api/v2/depu...,Abou Anni,UNIÃO,https://dadosabertos.camara.leg.br/api/v2/part...,SP,56,https://www.camara.leg.br/internet/deputado/ba...,0,13:19:31,20220804,União Brasil,22.0


In [35]:
print(len(votos_partidos.index))

197189


In [36]:
facts_table = votos_partidos.merge(votacoes_orgaos, how="left",on="idVotacao")
facts_table['facts_key'] = facts_table.index
facts_table.head(1)

,idVotacao,uriVotacao,dataHoraVoto,voto,deputado_id,deputado_uri,deputado_nome,deputado_siglaPartido,deputado_uriPartido,deputado_siglaUf,...,idOrgao,descricaoVotacao,aprovacao,votacoes_key,data,sigla,apelido,tipoOrgao,orgao_key,facts_key
0,105464-319,https://dadosabertos.camara.leg.br/api/v2/vota...,2022-08-04T13:19:31,Não,204521,https://dadosabertos.camara.leg.br/api/v2/depu...,Abou Anni,UNIÃO,https://dadosabertos.camara.leg.br/api/v2/part...,SP,...,180,Rejeitado o Destaque. Sim: 122; não: 246; abst...,0.0,2028,20220804,PLEN,Plenário,Plenário Virtual,1,0


In [37]:
print(len(facts_table.index))

197189


#### Step 3: Load data into (a different) MySQL (database)

In [38]:
createDatabase(mysql_db_dst)

In [39]:
tables = [("votacoes_dim",votacoes_dim,"votacoes_key"), ("orgaos_dim",orgaos_dim,"orgao_key"),
          ("votos_dim",votos_dim,"votos_key"),("partidos_dim",partidos_dim,"partidos_key"),
          ("facts_table",facts_table,"facts_key")]
for name,table,key in tables:
    set_dataframe2(table,mysql_db_dst,name,"insert",key)

### Check My Work
In this section I will:
(1) SELECT data from 5 tables(4 dimensions and the fact table)
(2) Perform an aggregation function to see the voting result for each vote that was taken

In [40]:
# Write Queries
# SELECT Queries
selectVotacoesDimQuery = "SELECT * FROM votacoes_dim"
selectOrgaosDimQuery = "SELECT * FROM orgaos_dim"
selectVotosDimQuery = "SELECT * FROM votos_dim"
selectPartidosDimQuery = "SELECT * FROM partidos_dim"
selectFactsTableDimQuery = "SELECT * FROM facts_table"

# Aggregation function
aggregationQuery = '''
SELECT idVotacao, 
        descricaoVotacao, 
        `data`,
        SUM(voto="Sim") as "sim",
        SUM(voto="Não") as "nao", 
        SUM(voto="Abstencão") as "abstencao" 
            FROM facts_table 
            GROUP BY idVotacao,
                    descricaoVotacao,
                    `data`
            ORDER BY `data` DESC;
'''
#Translations for this query:
#idVotacao = idVote
#descricaoVotacao = voteDescription
#Sim = Yes
#Não = No
#Abstencão = Abstentions

In [41]:
get_dataframe(user_id,pwd,host_name,mysql_db_dst, selectVotacoesDimQuery).head(1)

,idVotacao,idOrgao,descricaoVotacao,aprovacao,votacoes_key,data
0,2307143-25,2009,Aprovado o Relatório Prévio.,1.0,0,20221214


In [42]:
get_dataframe(user_id,pwd,host_name,mysql_db_dst, selectOrgaosDimQuery).head(1)

,idOrgao,sigla,apelido,tipoOrgao,orgao_key
0,4,MESA,Mesa Diretora,Comissão Diretora,0


In [43]:
get_dataframe(user_id,pwd,host_name,mysql_db_dst, selectVotosDimQuery).head(1)

,idVotacao,uriVotacao,dataHoraVoto,voto,deputado_id,deputado_uri,deputado_nome,deputado_siglaPartido,deputado_uriPartido,deputado_siglaUf,deputado_idLegislatura,deputado_urlFoto,votos_key,horaVoto,dataVoto
0,105464-319,https://dadosabertos.camara.leg.br/api/v2/vota...,2022-08-04T13:19:31,Não,204521,https://dadosabertos.camara.leg.br/api/v2/depu...,Abou Anni,UNIÃO,https://dadosabertos.camara.leg.br/api/v2/part...,SP,56,https://www.camara.leg.br/internet/deputado/ba...,0,13:19:31,20220804


In [44]:
get_dataframe(user_id,pwd,host_name,mysql_db_dst, selectPartidosDimQuery).head(1)

,deputado_nomePartido,sigla,partidos_key
0,Avante,AVANTE,0


In [45]:
get_dataframe(user_id,pwd,host_name,mysql_db_dst, selectFactsTableDimQuery).head(1)

,idVotacao,uriVotacao,dataHoraVoto,voto,deputado_id,deputado_uri,deputado_nome,deputado_siglaPartido,deputado_uriPartido,deputado_siglaUf,...,idOrgao,descricaoVotacao,aprovacao,votacoes_key,data,sigla,apelido,tipoOrgao,orgao_key,facts_key
0,105464-319,https://dadosabertos.camara.leg.br/api/v2/vota...,2022-08-04T13:19:31,Não,204521,https://dadosabertos.camara.leg.br/api/v2/depu...,Abou Anni,UNIÃO,https://dadosabertos.camara.leg.br/api/v2/part...,SP,...,180,Rejeitado o Destaque. Sim: 122; não: 246; abst...,0.0,2028,20220804,PLEN,Plenário,Plenário Virtual,1,0


In [46]:
# kept descricaoVoto since it says the score of the vote, so I can check my aggregation function is working correctly
get_dataframe(user_id,pwd,host_name,mysql_db_dst, aggregationQuery).head(1)

,idVotacao,descricaoVotacao,data,sim,nao,abstencao
0,2194899-116,Rejeitado o Requerimento. Sim: 144; não: 326; ...,20221221,144.0,326.0,1.0
